# **prepare current working directory (cwd)**

In [ ]:
%%bash
# remove everything except this notebook
shopt -s extglob
rm -rf !("notebook.ipynb") 
# to test any error in the terminal, use:
# ipython3 -c "%run notebook.ipynb"

In [ ]:
# import tools
import sys, os; sys.path.append(os.path.realpath(os.path.join(os.getcwd(), "../../")))
from tools import *
# while updating modules
# import importlib; importlib.reload(visualize_molecules)
# import importlib; importlib.reload(utils)

import ipywidgets
import glob
import ast
from tqdm import tqdm
import pandas as pd

import rdkit
from rdkit import Chem
import meeko
from meeko import MoleculePreparation
from meeko import PDBQTMolecule
from meeko import RDKitMolCreate
from pymol import cmd
from vina import Vina

import shutil
from pdbfixer import PDBFixer
from openmm.app import PDBFile

# Global variables
dirs_dict = {
             # 'raw_ligands':'./raw_ligands/', 
             'prepared_ligands':'../../../E*mutations/data/prev_prepared_molecules/prev_prepared_ligands/', 
             # 'raw_receptors':'./raw_receptors/', 
             'prepared_receptors':'../../../E*mutations/data/prev_prepared_molecules/prev_prepared_receptors/', 
             'vina_scoring':'../04_dock_from_prev_prepared_mols/vina_scoring/', 
             'vina_docking':'../04_dock_from_prev_prepared_mols/vina_docking/'
            }
vina_seed = 1

# **visualize**

for comments and more info about this section, take a look at notebook in directory 'example06 - visualize molecules'

## visualize ligands

ref: https://birdlet.github.io/2019/10/02/py3dmol_example/

In [ ]:
# List prepared molecules
# receptor_paths, receptor_names = directory_scraping.list_files_in_dir(directory=dirs_dict['prepared_receptors'], search_pattern='*.pdb*')
ligand_paths, ligand_names = directory_scraping.list_files_in_dir(directory=dirs_dict['prepared_ligands'], search_pattern='*.pdb*')
mols = [] 
for ligand_path in ligand_paths:
    try:
        mol = parse_molecules.MolFromPDBQTBlock(block=ligand_path,sanitize=False,removeHs=False)
        mols.append(mol)
    except Exception as e:
        print("An exception occurred")
        print("exception", e)

In [ ]:
def ligand_selector(idx, style, show_surface):
    mol = mols[idx-1]
    return visualize_molecules.show_3d_ligand(mol, style=style, surface=show_surface).show()

# Interactive widgets
ipywidgets.interact(ligand_selector, 
         idx = ipywidgets.IntSlider(min=1,max=len(mols), step=1),
         style = ipywidgets.Dropdown(options=['line', 'stick', 'sphere'], value='stick', description='Style:'),
         show_surface = ipywidgets.Checkbox(value=False, description='show surface')
        )

## visualize receptors with pockets

In [ ]:
# List prepared molecules
receptor_paths, receptor_names = directory_scraping.list_files_in_dir(directory=dirs_dict['prepared_receptors'], search_pattern='*.pdb*')

In [ ]:
def receptor_selector(idx, show_pockets, show_drug_score, show_surface):
    receptor_path, receptor_name = receptor_paths[idx-1], receptor_names[idx-1]
    pocket_paths = sorted(glob.glob(dirs_dict['prepared_receptors'] + receptor_name + '_out/*.pqr'))
    pockets_data = pd.read_csv(dirs_dict['prepared_receptors']+'fp_' + receptor_name + '.csv',sep=' ',index_col=False).set_index('cav_id')
    visualize_molecules.show_3d_receptor(receptor_path, receptor_name, 
                            show_pockets=show_pockets, show_drug_score=show_drug_score, show_surface=show_surface, 
                            pocket_paths=pocket_paths, pockets_data=pockets_data).show()
    print("receptor_name:", receptor_name)

# Interactive widgets
ipywidgets.interact(receptor_selector, 
                    idx = ipywidgets.IntSlider(min=1,max=len(receptor_paths), step=1),
                    # style = ipywidgets.Dropdown(options=['line', 'stick', 'sphere'], value='stick', description='Style:'),
                    show_pockets = ipywidgets.Checkbox(value=True, description='show pockets'),
                    show_drug_score = ipywidgets.Checkbox(value=False, description='show pockets drug_score'),
                    show_surface = ipywidgets.Checkbox(value=False, description='show surface')
        )

## visualize scorings

In [ ]:
# IMPORTANT: CHANGE THIS CELL AS YOU PREFER
# filter the vina_scores according to your needs
vina_score_max = -1
max_num_dockings = 20

vina_scores = pd.read_csv(dirs_dict['vina_scoring']+'vina_scores.csv',sep='\t')
vina_scores_filtered = vina_scores[vina_scores['vina_score_after_min'] <= vina_score_max] \
                        .sort_values(by='vina_score_after_min', ascending=True) \
                        [0:max_num_dockings]
vina_scores_filtered.rename(columns={'vina_score_box_size': 'vina_box_size'}, inplace=True)
vina_scores_filtered

In [ ]:
def vina_scoring_selector(show_pockets, show_drug_score, show_surface, show_vina_box, ligand_style, idx):
    # Receptor related variables
    scoring_row = vina_scores_filtered.iloc[idx]
    receptor_name = scoring_row['receptor_name']
    receptor_path = dirs_dict['prepared_receptors'] + receptor_name + '.pdbqt'
    pocket_paths = sorted(glob.glob(dirs_dict['prepared_receptors'] + receptor_name + '_out/*.pqr'))
    pockets_data = pd.read_csv(dirs_dict['prepared_receptors']+'fp_' + receptor_name + '.csv',sep=' ',index_col=False).set_index('cav_id')
    # Ligands related variables
    # pdbqt_mol = PDBQTMolecule.from_file(output_path, skip_typing=True)
    # rdkitmol_list = RDKitMolCreate.from_pdbqt_mol(pdbqt_mol)
    output_path = dirs_dict['vina_scoring'] + os.path.basename(scoring_row['pose_after_min_path'])
    !mk_export.py {output_path} -o aux_rdkitmol_list.sdf
    rdkitmol_list = [x for x in Chem.SDMolSupplier('aux_rdkitmol_list.sdf')]
    output_path = dirs_dict['vina_scoring'] + os.path.basename(scoring_row['pose_before_min_path'])
    !mk_export.py {output_path} -o aux_rdkitmol_list.sdf
    [rdkitmol_list.append(x) for x in Chem.SDMolSupplier('aux_rdkitmol_list.sdf')]
    # utils.pdbqt_to_sdf(pdbqt_file=output_path, output=dirs_dict['vina_scoring']+'aux_vina_out_sdf.sdf', include_data=False)
    # rdkitmol_list = Chem.SDMolSupplier(dirs_dict['vina_scoring']+'aux_vina_out_sdf.sdf')
    
    # 3d plot
    visualize_molecules.show_3d_vina_score_dock(receptor_path, receptor_name, show_pockets, show_drug_score, show_surface, 
                                                show_vina_box, ligand_style, rdkitmol_list, pocket_paths, pockets_data, 
                                                scoring_row, False).show()
    
    # Display selected row from dataframe
    display(vina_scores_filtered.iloc[[idx]])

# Interactive widgets
ipywidgets.interact(vina_scoring_selector, 
                    idx = ipywidgets.IntSlider(min=0,max=len(vina_scores_filtered)-1, step=1),
                    show_pockets = ipywidgets.Checkbox(value=False, description='show pockets'),
                    show_drug_score = ipywidgets.Checkbox(value=False, description='show pockets drug_score'),
                    show_surface = ipywidgets.Checkbox(value=False, description='show surface'),
                    show_vina_box = ipywidgets.Checkbox(value=False, description='show box used for vina'),
                    ligand_style = ipywidgets.Dropdown(options=['line', 'stick', 'sphere'], value='stick', description='ligand style:'),
        )

## visualize dockings

In [ ]:
vina_dockings = pd.read_csv(dirs_dict['vina_docking']+'vina_dockings.csv',sep='\t')
vina_dockings.rename(columns={'vina_docking_box_size': 'vina_box_size'}, inplace=True)
vina_dockings

In [ ]:
import importlib; importlib.reload(visualize_molecules)

def vina_docking_selector(show_pockets, show_only_target_pocket, show_drug_score, show_surface, show_vina_box, show_only_best_pose, ligand_style, idx):
    # Receptor related variables
    docking_row = vina_dockings.iloc[idx]
    receptor_name = docking_row['receptor_name']
    receptor_path = dirs_dict['prepared_receptors'] + receptor_name + '.pdbqt'
    pocket_paths = sorted(glob.glob(dirs_dict['prepared_receptors'] + receptor_name + '_out/*.pqr'))
    pockets_data = pd.read_csv(dirs_dict['prepared_receptors']+'fp_' + receptor_name + '.csv',sep=' ',index_col=False).set_index('cav_id')
    # Ligands related variables
    output_path = dirs_dict['vina_docking'] + os.path.basename(docking_row['output_path'])
    # pdbqt_mol = PDBQTMolecule.from_file(output_path, skip_typing=True)
    # rdkitmol_list = RDKitMolCreate.from_pdbqt_mol(pdbqt_mol)
    # from: https://github.com/forlilab/Meeko
    !mk_export.py {output_path} -o aux_rdkitmol_list.sdf
    rdkitmol_list = [x for x in Chem.SDMolSupplier('aux_rdkitmol_list.sdf')]
    # utils.pdbqt_to_sdf(pdbqt_file=output_path, output=dirs_dict['vina_docking']+'aux_vina_out_sdf.sdf')
    # rdkitmol_list = Chem.SDMolSupplier(dirs_dict['vina_docking']+'aux_vina_out_sdf.sdf')
    pocket_id = None
    if show_only_target_pocket:
        pocket_id = docking_row['pocket_id']
    
    # 3d plot
    visualize_molecules.show_3d_vina_score_dock(receptor_path, receptor_name, show_pockets, show_drug_score, show_surface, 
                                                show_vina_box, ligand_style, rdkitmol_list, pocket_paths, pockets_data, 
                                                docking_row, show_only_best_pose, pocket_id).show()
    
    # Display selected row from dataframe
    display(vina_dockings.iloc[[idx]])

# Interactive widgets
ipywidgets.interact(vina_docking_selector, 
                    idx = ipywidgets.IntSlider(min=0,max=len(vina_dockings)-1, step=1),
                    show_pockets = ipywidgets.Checkbox(value=False, description='show pockets'),
                    show_only_target_pocket = ipywidgets.Checkbox(value=False, description='show_only_target_pocket'),
                    show_drug_score = ipywidgets.Checkbox(value=False, description='show pockets drug_score'),
                    show_surface = ipywidgets.Checkbox(value=False, description='show surface'),
                    show_vina_box = ipywidgets.Checkbox(value=False, description='show box used for vina'),
                    show_only_best_pose = ipywidgets.Checkbox(value=False, description='show only the best pose'),
                    ligand_style = ipywidgets.Dropdown(options=['line', 'stick', 'sphere'], value='stick', description='ligand style:'),
        )